<img src="../qbies.png" style="width: 100px;padding-right:5px;padding-top:1px;padding-left:5px;" align="left"/>
 <p style="padding-left:125px;padding-top: 30px";><font size="+2"><b> Practical Guidance - Iterators</b></font><p>

## Accumulating Iterators - Advanced Topic

[Accumulation](https://code.kx.com/q/ref/accumulators/) iterators execute repeatedly over the values returned from calling the modified function. 


There are two accumulator iterations in kdb+/q, both of which operate in the exact same fashion, excepting their return value. These functions are: 
* Scan (`\`) 
* Over (`/`)

The difference between the two is that when scan (`\`) returns the intermediate values associated with each execution, while over (`/`) returns just the final value. Over requires less memory. 


**These iterators aren't part of the curriculum for the Kx Fundamentals course and are instead introduced in the Kx Advanced course, the below serves as an introduction for the eager attendee.**


### Scan

The iterator scan, denoted by `\` (backslash) or the reserved word `scan`, has many behaviours, one of which is to  modify a dyadic or monadic function.

When scan acts upon a dyadic function, it creates a uniform monadic function. In mathematical notation, the output of a scan function is as follows:
\begin{align}
y_0=x_0 
,y_n=f(y_{n-1};x_n)
\end{align}

Where $x_n$ is the $n^{th}$ element of the input list, $y_n$ is the $n^{th}$ element of the returned output list, and $f$ is our function. 

Examples are perhaps the most illustrative: 

In [7]:
(+\)1 2 3 4 5                       //solution is (1) (1)+2 (1+2)+3 (1+2+3)+4 (1+2+3+4)+5

1 3 6 10 15


Lets break down each step of how we produce our output: 
> 1. We start our return with the first item of our input `1`. 
* After this we apply `+` with inputs `+[1;2]` to return `3`. 
*  Next, use our previous return `3`as the first input and `3` as the second input to evaluate `+[3;3]` as `6`
* Next, use our previous return `6` as the first input and `4` as the second input to evaluate `+[6;4]` as `10`
* Finally, use our previous return `10` as the first input and `5` as the second input to evaluate `+[10;5]` as `15`

If we wanted to use the keyword `scan` instead we use the below syntax.

In [8]:
(+) scan 1 2 3 4 5                  //note that this is the same as sums

1 3 6 10 15


Again, this consecutive summing is such a popular operation that there is a keyword dedicated to it - [`sums`](https://code.kx.com/q/ref/sum/). 

In [9]:
sums 1 2 3 4 5 
sums                                //we can see the definition of sums

1 3 6 10 15


+\


If we wanted to append one element of a list to the next to a list in each iteration, we can do so using scan: 

In [10]:
(,\)(1 +til 5)

,1
1 2
1 2 3
1 2 3 4
1 2 3 4 5


###### Worked Example 

This iterator isn't solely reserved for use with inbuilt function, it can also be applied to our functions. 

Lets work through an example of compounding interest on money in a savings account:

**Given**: savings account balance, interest rate

**Find**: a function that will tell us our yearly return

In [11]:
yearlyReturn:{[savingsBalance;interest] rate: 1+0.01*interest;  //multiplying is a "cheaper" operation than division
                        savingsBalance*rate}  
yearlyReturn[10000;1.2]   //account balance at the end of one year with the at 1.2% savings interest rate 

10120f


Let's start with an estimation of the next few years interest rates, and for comparison purposes lets have a look at the S&P annual change over the last five years: 

In [12]:
interestRates: 1 1 1.25 2 1.6 1.5            //2016, 2017, 2018, 2019, 2020 YTD per Ally bank - https://www.consumerismcommentary.com/rates/
last5yearsSnP: 9.54 19.42 -6.24 28.88 -5.77  //2016, 2017, 2018, 2019, 2020 YTD - https://www.macrotrends.net/2526/sp-500-historical-annual-returns

We can calculate the compounding interest on this by providing a starting account balance and then supplying the subsequent rates:

In [13]:
(yearlyReturn\) 10000f,interestRates   //compounding at saving rate over last 5 years
(yearlyReturn\) 10000f,last5yearsSnP   //compounding at S&P return over last 5 years

10000 10100 10201 10328.51 10535.08 10703.64 10864.2


10000 10954 13081.27 12265 15807.13 14895.06


Compounding is important because we see at the end we have quite a big difference!

### Over

The adverb `over`, denoted by `/` (forward slash) or the reserved word `over`, operates in the same way as scan, but `over` only returns the **last item** that `scan` produces. 

In [14]:
(+/)1 2 3 4 5
(+) over 1 2 3 4 5

15


15


In [15]:
sum 1 2 3 4 5
(,) over (1 +til 5)

15


1 2 3 4 5


In [16]:
(yearlyReturn/)[10000f;interestRates]
(yearlyReturn/)[10000f;last5yearsSnP]

10864.2


14895.06


<img src="../qbies.png" style="width: 50px;padding-right:5px;padding-top:5px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> If puzzled by the result of using <code>over</code>, try replacing it with <code>scan</code> and examining the intermediate results!</i></p>

## Iterators versus Loops (do and while)

The control words [`do`](https://code.kx.com/q/ref/do/) and [`while`](https://code.kx.com/q/ref/while/) allow q programmers to write explicit loops. However loops/iteration can almost always be defined using iterators, yielding code that is shorter, faster and less prone to error.

Often the implementation is relatively easy, using Each, Each Left and Each Right to cycle through a list and amend its items.

Suppose we wanted to check if either of the integers 2 or 3 are present in some lists. This can be achieved with a while loop.

In [17]:
show m:(1 2 3;3 4 5;4 5 6)   / three lists
{i:0;a:();while[i<count x;a,:enlist any 2 3 in x[i];i+:1];a} m
\t:100000 {i:0;a:();while[i<count x;a,:enlist any 2 3 in x[i];i+:1];a} m

1 2 3
3 4 5
4 5 6


110b


284


However, iterators allow neater, more efficient code; easier to read and cheaper to maintain.

In [18]:
any each 2 3 in/: m

110b


In [19]:
\t:100000 any each 2 3 in/: m

178


Similarly we can use the Over iterator to deal easily with situations which would be handled by loops in C-like languages.

Suppose you wanted to join several tables.

In [36]:
//Create a list of tables, of random length
show tt:{1!flip(`sym;`$"pr",x;`$"vol",x)!(`a`b`c;3?50.0;3?100)}each string til 2+rand 10

(+(,`sym)!,`a`b`c)!+`pr0`vol0!(31.65205 28.76346 0.4505872;24 51 58)
(+(,`sym)!,`a`b`c)!+`pr1`vol1!(5.876775 46.17193 20.07968;91 44 15)
(+(,`sym)!,`a`b`c)!+`pr2`vol2!(10.43307 49.53558 28.97401;82 90 66)
(+(,`sym)!,`a`b`c)!+`pr3`vol3!(29.94584 2.440864 45.03495;93 5 58)


In [37]:
//one table 
tt[0] 

sym| pr0       vol0
---| --------------
a  | 31.65205  24  
b  | 28.76346  51  
c  | 0.4505872 58  


In [38]:
//Join the tables using a while loop
{a:([]sym:`a`b`c);i:0;while[i<count[x];a:a lj x[i];i+:1];a}tt

sym pr0       vol0 pr1      vol1 pr2      vol2 pr3      vol3
------------------------------------------------------------
a   31.65205  24   5.876775 91   10.43307 82   29.94584 93  
b   28.76346  51   46.17193 44   49.53558 90   2.440864 5   
c   0.4505872 58   20.07968 15   28.97401 66   45.03495 58  


In [39]:
//Join the tables using Over
0!(lj/)tt
\c 200 100 

sym pr0       vol0 pr1      vol1 pr2      vol2 pr3      vol3
------------------------------------------------------------
a   31.65205  24   5.876775 91   10.43307 82   29.94584 93  
b   28.76346  51   46.17193 44   49.53558 90   2.440864 5   
c   0.4505872 58   20.07968 15   28.97401 66   45.03495 58  


<img src="../qbies.png" style="width: 50px;padding-right:5px;padding-top:5px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Write loops only when you can find no solution using iterators.</i></p>

## Combining iterators

We can calculate [Pascal’s Triangle](https://en.wikipedia.org/wiki/Pascal%27s_triangle) using Scan and Each Prior.

We already have a sufficient grasp of the accumulators to see the Triangle immediately as successive results from some use of Scan. We need only a function to define one row of the Triangle in terms of the row above it. That could hardly be simpler. Each row is derived from its parent by summing adjacent pairs of items.



In [6]:
7 {(+)prior x,0}\ 1

1
1 1
1 2 1
1 3 3 1
1 4 6 4 1
1 5 10 10 5 1
1 6 15 20 15 6 1
1 7 21 35 35 21 7 1


Notice that the last expression gave us eight rows of the Triangle, not seven. The first item of the result was the original argument, followed by the results of seven successive evaluations.

If the left argument of the derived function is zero, there will be no evaluations. The original argument will still be returned as the first (and only) item of the result. It doesn’t even have to be in the domain of the value.